# Análise Exploratória de Dados - Detecção de Phishing

Este notebook contém uma análise exploratória dos dados utilizados no projeto de detecção de phishing. Vamos explorar o dataset, entender a distribuição dos dados, identificar padrões e características importantes para a classificação de URLs maliciosas.

## 1. Importação das Bibliotecas

Vamos importar as bibliotecas necessárias para nossa análise:

In [ ]:
# Importar bibliotecas para análise de dados
import pandas as pd
import numpy as np

# Bibliotecas para visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações para visualização
plt.style.use('ggplot')
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

# Para análise estatística
from scipy import stats

# Para ignorar warnings
import warnings
warnings.filterwarnings('ignore')

## 2. Carregamento dos Dados

Vamos carregar o dataset de phishing para análise:

In [ ]:
# Definir o caminho do arquivo
dataset_path = "../data/phishing_dataset_CIS.csv"

# Carregar o dataset
try:
    df = pd.read_csv(dataset_path)
    print(f"Dataset carregado com sucesso. Formato: {df.shape}")
except FileNotFoundError:
    print(f"Arquivo não encontrado: {dataset_path}")
    # Tentar caminho alternativo
    alternative_path = "../data/phishing_dataset_CIS.csv"
    try:
        df = pd.read_csv(alternative_path)
        print(f"Dataset carregado do caminho alternativo. Formato: {df.shape}")
    except FileNotFoundError:
        print(f"Arquivo não encontrado também em: {alternative_path}")
        print("Por favor, verifique o caminho correto do arquivo de dados.")
        # Criar um DataFrame vazio para continuar o notebook
        df = pd.DataFrame()

In [ ]:
# Verificar as primeiras linhas do dataset
if not df.empty:
    print("\n=== Primeiras 5 linhas do dataset ===")
    display(df.head())
else:
    print("Dataset vazio, não é possível mostrar as primeiras linhas.")

## 3. Informações Gerais do Dataset

Vamos analisar as informações gerais do dataset, como tipos de dados, estatísticas básicas e valores nulos:

In [ ]:
# Informações básicas do dataset
if not df.empty:
    # Informações sobre os tipos de dados
    print("=== Tipos de dados ===")
    print(df.dtypes)
    
    # Estatísticas descritivas
    print("\n=== Estatísticas descritivas ===")
    display(df.describe())
    
    # Informações sobre valores nulos
    print("\n=== Valores nulos por coluna ===")
    null_counts = df.isnull().sum()
    null_percentages = (df.isnull().mean() * 100).round(2)
    null_info = pd.DataFrame({
        'Contagem de Nulos': null_counts,
        '% de Nulos': null_percentages
    })
    display(null_info)
else:
    print("Dataset vazio, não é possível mostrar as informações gerais.")

## 4. Análise da Variável Alvo (Target)

Vamos analisar a distribuição da variável alvo (phishing) para entender o balanceamento das classes:

In [ ]:
if not df.empty:
    # Verificar se existe uma coluna 'phishing' ou similar no dataset
    target_col = None
    potential_target_cols = ['phishing', 'target', 'label', 'is_phishing', 'class']
    
    for col in potential_target_cols:
        if col in df.columns:
            target_col = col
            break
    
    if target_col:
        # Distribuição da variável alvo
        print(f"=== Distribuição da variável alvo '{target_col}' ===")
        target_counts = df[target_col].value_counts()
        target_percentages = df[target_col].value_counts(normalize=True).mul(100).round(2)
        
        target_distribution = pd.DataFrame({
            'Contagem': target_counts,
            'Porcentagem (%)': target_percentages
        })
        display(target_distribution)
        
        # Visualizar a distribuição
        plt.figure(figsize=(10, 6))
        sns.countplot(x=target_col, data=df)
        plt.title(f'Distribuição da Variável Alvo ({target_col})')
        plt.xlabel(target_col)
        plt.ylabel('Contagem')
        plt.xticks(rotation=0)
        
        # Adicionar rótulos com as contagens
        for i, count in enumerate(target_counts):
            plt.text(i, count + 10, f'{count} ({target_percentages.iloc[i]}%)', 
                    ha='center', va='bottom', fontweight='bold')
        
        plt.tight_layout()
        plt.show()
    else:
        print("Não foi possível identificar a coluna alvo (target) no dataset.")
else:
    print("Dataset vazio, não é possível analisar a variável alvo.")

## 5. Análise de Características Numéricas

Vamos analisar as características numéricas do dataset, visualizando suas distribuições e correlações:

In [ ]:
if not df.empty:
    # Identificar colunas numéricas
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    
    # Excluir a coluna alvo se estiver presente
    target_cols = ['phishing', 'target', 'label', 'is_phishing', 'class']
    numeric_features = [col for col in numeric_cols if col not in target_cols]
    
    if numeric_features:
        print(f"=== Características numéricas: {len(numeric_features)} ===")
        print(f"Colunas: {numeric_features[:10]}...")
        if len(numeric_features) > 10:
            print(f"(Mostrando apenas as primeiras 10 de {len(numeric_features)} colunas)")
        
        # Selecionar as 5 primeiras características numéricas para visualização detalhada
        visual_features = numeric_features[:5]
        
        # Histogramas das características numéricas selecionadas
        plt.figure(figsize=(15, 10))
        for i, feature in enumerate(visual_features):
            plt.subplot(2, 3, i+1)
            sns.histplot(df[feature], kde=True)
            plt.title(f'Distribuição de {feature}')
            plt.xlabel(feature)
            plt.ylabel('Contagem')
        plt.tight_layout()
        plt.show()
        
        # Boxplots para identificar outliers
        plt.figure(figsize=(15, 10))
        for i, feature in enumerate(visual_features):
            plt.subplot(2, 3, i+1)
            sns.boxplot(y=df[feature])
            plt.title(f'Boxplot de {feature}')
            plt.ylabel(feature)
        plt.tight_layout()
        plt.show()
        
        # Matriz de correlação entre as características numéricas
        # Limitando para as primeiras 15 características para melhor visualização
        correlation_features = numeric_features[:15]
        if correlation_features:
            correlation_matrix = df[correlation_features].corr()
            
            plt.figure(figsize=(12, 10))
            sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
            plt.title('Matriz de Correlação das Características Numéricas')
            plt.tight_layout()
            plt.show()
    else:
        print("Não foram encontradas características numéricas no dataset.")
else:
    print("Dataset vazio, não é possível analisar características numéricas.")

## 6. Análise Bivariada - Relação entre Features e Target

Vamos analisar como as características se relacionam com a variável alvo:

In [ ]:
if not df.empty:
    # Verificar se existe uma coluna target
    target_col = None
    potential_target_cols = ['phishing', 'target', 'label', 'is_phishing', 'class']
    
    for col in potential_target_cols:
        if col in df.columns:
            target_col = col
            break
    
    if target_col and len(numeric_features) > 0:
        # Selecionar as 5 características numéricas mais importantes para visualização
        top_features = numeric_features[:5]
        
        # Visualizar a relação entre as top features e a variável alvo
        print(f"=== Análise da relação entre as principais características e '{target_col}' ===")
        
        # Boxplots das características por classe
        plt.figure(figsize=(15, 12))
        for i, feature in enumerate(top_features):
            plt.subplot(2, 3, i+1)
            sns.boxplot(x=target_col, y=feature, data=df)
            plt.title(f'{feature} por {target_col}')
            plt.xlabel(target_col)
            plt.ylabel(feature)
        plt.tight_layout()
        plt.show()
        
        # Violinplots para melhor visualização da distribuição
        plt.figure(figsize=(15, 12))
        for i, feature in enumerate(top_features):
            plt.subplot(2, 3, i+1)
            sns.violinplot(x=target_col, y=feature, data=df)
            plt.title(f'Distribuição de {feature} por {target_col}')
            plt.xlabel(target_col)
            plt.ylabel(feature)
        plt.tight_layout()
        plt.show()
        
        # Calcular importância das características usando correlação com o target
        if target_col in df.columns:
            correlations = df[numeric_features].corrwith(df[target_col]).abs().sort_values(ascending=False)
            
            print("\n=== Correlação das features numéricas com o target ===")
            correlations_df = pd.DataFrame({
                'Feature': correlations.index,
                'Correlação Absoluta': correlations.values
            })
            display(correlations_df.head(10))
            
            # Visualizar as correlações
            plt.figure(figsize=(12, 8))
            sns.barplot(x='Correlação Absoluta', y='Feature', data=correlations_df.head(10))
            plt.title('Top 10 Features com Maior Correlação Absoluta com o Target')
            plt.tight_layout()
            plt.show()
    else:
        if not target_col:
            print("Não foi possível identificar a coluna alvo (target) no dataset.")
        if len(numeric_features) == 0:
            print("Não foram encontradas características numéricas para análise bivariada.")
else:
    print("Dataset vazio, não é possível realizar análise bivariada.")

## 7. Conclusões e Insights

Com base na análise exploratória realizada, podemos destacar os seguintes insights:

1. **Distribuição das Classes**: Analisamos o balanceamento entre URLs legítimas e URLs de phishing no dataset.

2. **Características mais Relevantes**: Identificamos as características com maior correlação com a variável alvo, que são potencialmente mais úteis para a detecção de phishing.

3. **Padrões nas Features**: Observamos como as características se comportam em relação às classes (phishing vs. legítimo), destacando diferenças importantes que podem ser exploradas por modelos de machine learning.

4. **Outliers e Valores Extremos**: Identificamos a presença de outliers em algumas características, o que pode afetar o desempenho de modelos de machine learning se não forem tratados adequadamente.

5. **Correlações entre Features**: Analisamos a matriz de correlação para identificar características redundantes que podem ser removidas para simplificar o modelo sem perda significativa de informação.

Este notebook fornece uma base sólida para entender o dataset e guiar decisões de pré-processamento e modelagem para o projeto de detecção de phishing.

In [ ]:
# Próximos passos para o projeto
print("=== Próximos Passos ===")
print("1. Pré-processamento dos dados (tratamento de valores nulos, normalização, etc.)")
print("2. Feature engineering (criar novas características, selecionar as mais relevantes)")
print("3. Dividir o dataset em conjuntos de treino e teste")
print("4. Treinar modelos de machine learning e avaliar seu desempenho")
print("5. Otimizar hiperparâmetros do modelo com melhor desempenho")
print("6. Implementar o modelo final para detecção de phishing")